<h1>Coleta e Transformação de Dados de API</h1>
Éverton Bin | Desafio 2 - Bootcamp Cientista de Dados IGTI

In [1]:
# Importando pacotes:
import json
import requests
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Conectando com a API:
response = requests.get('https://api-coleta-dados.herokuapp.com/api/pessoas')

In [3]:
# Verificando se a conexão encontrou a página:
response.status_code

200

In [4]:
# Salvando o conteúdo em um objeto 'json':
api_content = json.loads(response.content)

In [5]:
# Verificando as 'features' do primeiro registro:
api_content[0]

{'nome': 'Silvana Hadassa',
 'idade': None,
 'data_nasc': '12/06/1964',
 'sexo': 'Feminino',
 'signo': 'Gêmeos',
 'cidade': 'Brasília',
 'estado': 'DF',
 'altura': '1,67',
 'peso': 50,
 'tipo_sanguineo': 'A+',
 'cor': 'vermelho'}

In [6]:
# Convertendo o objeto tipo json para um df do Pandas para fazer o tratamento de dados:
content_df = pd.DataFrame.from_dict(api_content, orient = 'columns')

In [7]:
# Verificando os primeiros registros do df:
content_df.head()

,nome,idade,data_nasc,sexo,signo,cidade,estado,altura,peso,tipo_sanguineo,cor
0,Silvana Hadassa,NaN,12/06/1964,Feminino,Gêmeos,Brasília,DF,"1,67",50,A+,vermelho
1,Thales Kaique,27.0,10/12/1993,Masculino,Sagitário,Campo Grande,MS,"1,63",110,B+,laranja
2,Henry Nathan,29.0,21/09/1991,Masculino,Virgem,Macapá,AP,"1,86",106,AB-,verde
3,Lorenzo Vinicius,64.0,22/10/1956,Masculino,Libra,Contagem,MG,"1,87",100,O+,vermelho
4,Cauã Henrique,50.0,09/03,Masculino,Peixes,Rondonópolis,MT,"1,74",83,O-,vermelho


In [8]:
# Verificando o tipo dos dados:
content_df.dtypes

nome               object
idade             float64
data_nasc          object
sexo               object
signo              object
cidade             object
estado             object
altura             object
peso                int64
tipo_sanguineo     object
cor                object
dtype: object

In [9]:
# Atualizando as datas de nascimento sem ano declarado através das idades:
for i in range(0, len(content_df)):
    if len(content_df.data_nasc[i]) != 10:
        content_df.data_nasc[i] = content_df.data_nasc[i]+str(int(content_df.idade[i])-2020)

In [10]:
# Convertendo a coluna 'data_nasc' para o tipo data:
content_df['data_nasc'] = pd.to_datetime(content_df['data_nasc'], infer_datetime_format=True)

In [11]:
# Atualizando as idades pela data de nascimento (referência 31 de dezembro de 2020):
for i in range(0, len(content_df)):
    content_df.idade[i] = int(int(str(pd.Timestamp('2020-12-31') - content_df.data_nasc[i]).split(' ')[0])/365.2425)

In [12]:
# Transformando a data de nascimento para o tipo string:
content_df['data_nasc'] = content_df['data_nasc'].dt.strftime('%Y-%m-%d')

In [13]:
content_df.head()

,nome,idade,data_nasc,sexo,signo,cidade,estado,altura,peso,tipo_sanguineo,cor
0,Silvana Hadassa,56.0,1964-12-06,Feminino,Gêmeos,Brasília,DF,"1,67",50,A+,vermelho
1,Thales Kaique,27.0,1993-10-12,Masculino,Sagitário,Campo Grande,MS,"1,63",110,B+,laranja
2,Henry Nathan,29.0,1991-09-21,Masculino,Virgem,Macapá,AP,"1,86",106,AB-,verde
3,Lorenzo Vinicius,64.0,1956-10-22,Masculino,Libra,Contagem,MG,"1,87",100,O+,vermelho
4,Cauã Henrique,50.0,1970-09-03,Masculino,Peixes,Rondonópolis,MT,"1,74",83,O-,vermelho


<h1>Ingestão de Dados em DB MySQL</h1>

<p>Para a ingestão de dados, será respeitado o esquema criado para o banco de dados representado pela imagem que segue:</p>
<img src="der.png" />

In [14]:
# Criando variáveis numéricas que servirão de código id:
# Variável cor:
cor_array = content_df.cor.unique()
content_df['cor_id'] = 0
for i_cor in range(0, len(cor_array)):
    for i in range(0, len(content_df)):
        if content_df.cor[i] == cor_array[i_cor]:
            content_df.cor_id[i] = i_cor         

In [15]:
# Variável tiposanguineo:
tsang_array = content_df.tipo_sanguineo.unique()
content_df['tipo_sanguineo_id'] = 0
for i_tsang in range(0, len(tsang_array)):
    for i in range(0, len(content_df)):
        if content_df.tipo_sanguineo[i] == tsang_array[i_tsang]:
            content_df.tipo_sanguineo_id[i] = i_tsang

In [16]:
# Variável estado:
estado_array = content_df.estado.unique()
content_df['estado_id'] = 0
for i_est in range(0, len(estado_array)):
    for i in range(0, len(content_df)):
        if content_df.estado[i] == estado_array[i_est]:
            content_df.estado_id[i] = i_est

In [17]:
# Variável cidade:
cidade_array = content_df.cidade.unique()
content_df['cidade_id'] = 0
for i_cid in range(0, len(cidade_array)):
    for i in range(0, len(content_df)):
        if content_df.cidade[i] == cidade_array[i_cid]:
            content_df.cidade_id[i] = i_cid 

In [18]:
content_df.head()

,nome,idade,data_nasc,sexo,signo,cidade,estado,altura,peso,tipo_sanguineo,cor,cor_id,tipo_sanguineo_id,estado_id,cidade_id
0,Silvana Hadassa,56.0,1964-12-06,Feminino,Gêmeos,Brasília,DF,"1,67",50,A+,vermelho,0,0,0,0
1,Thales Kaique,27.0,1993-10-12,Masculino,Sagitário,Campo Grande,MS,"1,63",110,B+,laranja,1,1,1,1
2,Henry Nathan,29.0,1991-09-21,Masculino,Virgem,Macapá,AP,"1,86",106,AB-,verde,2,2,2,2
3,Lorenzo Vinicius,64.0,1956-10-22,Masculino,Libra,Contagem,MG,"1,87",100,O+,vermelho,0,3,3,3
4,Cauã Henrique,50.0,1970-09-03,Masculino,Peixes,Rondonópolis,MT,"1,74",83,O-,vermelho,0,4,4,4


In [19]:
# Importando pacote para conectar ao MySQL:
import mysql.connector

# Conectando ao db_desafio criado no MySQL:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='xxxxxxxx',
    database='db_desafio')

# Criando o cursor:
mycursor = mydb.cursor()

In [20]:
# Verificando as tabelas criadas no banco de dados: 
mycursor.execute("SHOW TABLES")

for tables in mycursor:
    print(tables)

('tb_cidade',)
('tb_cor',)
('tb_estado',)
('tb_pessoa',)
('tb_tiposanguineo',)


In [21]:
# Inserindo dados da tabela estado:
query_estado = "INSERT INTO tb_estado (id, sigla) VALUES (%s, %s)"
estado_id_array = content_df.estado_id.unique()

for i in range(0, len(estado_id_array)):
    data_estado = (int(estado_id_array[i]), estado_array[i])
    print(data_estado)
    mycursor.execute(query_estado, data_estado)
    
mydb.commit()

(0, 'DF')
(1, 'MS')
(2, 'AP')
(3, 'MG')
(4, 'MT')
(5, 'PE')
(6, 'AC')
(7, 'TO')
(8, 'AM')
(9, 'RO')
(10, 'GO')
(11, 'RS')
(12, 'RR')
(13, 'MA')
(14, 'ES')
(15, 'PI')
(16, 'PR')
(17, 'AL')
(18, 'SE')
(19, 'PA')
(20, 'BA')
(21, 'RJ')
(22, 'PB')
(23, 'SC')
(24, 'CE')
(25, 'RN')


In [22]:
# Inserindo dados da tabela cor:
query_cor = "INSERT INTO tb_cor (id, cor) VALUES (%s, %s)"
cor_id_array = content_df.cor_id.unique()

for i in range(0, len(cor_id_array)):
    data_cor = (int(cor_id_array[i]), cor_array[i])
    print(data_cor)
    mycursor.execute(query_cor, data_cor)
    
mydb.commit()

(0, 'vermelho')
(1, 'laranja')
(2, 'verde')
(3, 'preto')
(4, 'roxo')
(5, 'azul')
(6, 'amarelo')


In [23]:
# Inserindo dados da tabela tipo_sanguineo:
query_tsang = "INSERT INTO tb_tiposanguineo (id, tipo) VALUES (%s, %s)"
tsang_id_array = content_df.tipo_sanguineo_id.unique()

for i in range(0, len(tsang_id_array)):
    data_tsang = (int(tsang_id_array[i]), tsang_array[i])
    print(data_tsang)
    mycursor.execute(query_tsang, data_tsang)
    
mydb.commit()

(0, 'A+')
(1, 'B+')
(2, 'AB-')
(3, 'O+')
(4, 'O-')
(5, 'AB+')
(6, 'A-')
(7, 'B-')


In [24]:
# Criando um df para as cidades com seus respectivos id:
cidade_df = pd.DataFrame(cidade_array, columns = ['cidade'])
cidade_df['cidade_id'] = content_df.cidade_id.unique()
cidade_df.head()

,cidade,cidade_id
0,Brasília,0
1,Campo Grande,1
2,Macapá,2
3,Contagem,3
4,Rondonópolis,4


In [25]:
# Incluindo estado_id referente a cada uma das cidades:
cidade_df['estado_id'] = 0
for cid_i in range(0, len(cidade_df)):
    ind = 0
    while content_df.cidade.iloc[ind] != cidade_df.cidade.iloc[cid_i]:
        ind += 1
    else:
        cidade_df.estado_id.iloc[cid_i] = content_df.estado_id.iloc[ind]

In [26]:
# Inserindo os dados na tabela tb_cidade:
query_cidade = "INSERT INTO tb_cidade (id, nome, id_estado) VALUES (%s, %s, %s)"

for i in range(0, len(cidade_df)):
    data_cidade = (int(cidade_df.cidade_id.iloc[i]), cidade_df.cidade.iloc[i], int(cidade_df.estado_id.iloc[i]))
    print(data_cidade)
    mycursor.execute(query_cidade, data_cidade)
    
mydb.commit()

(0, 'Brasília', 0)
(1, 'Campo Grande', 1)
(2, 'Macapá', 2)
(3, 'Contagem', 3)
(4, 'Rondonópolis', 4)
(5, 'Olinda', 5)
(6, 'Rio Branco', 6)
(7, 'Araguaína', 7)
(8, 'Manaus', 8)
(9, 'Caruaru', 5)
(10, 'Porto Velho', 9)
(11, 'Planaltina', 10)
(12, 'Santa Cruz do Sul', 11)
(13, 'Porto Alegre', 11)
(14, 'Carpina', 5)
(15, 'Boa Vista', 12)
(16, 'Imperatriz', 13)
(17, 'São Luís', 13)
(18, 'Cacoal', 9)
(19, 'Cariacica', 14)
(20, 'Parnaíba', 15)
(21, 'Santana', 2)
(22, 'Ponta Grossa', 16)
(23, 'Arapiraca', 17)
(24, 'Águas Lindas de Goiás', 10)
(25, 'Cianorte', 16)
(26, 'Aracaju', 18)
(27, 'Belém', 19)
(28, 'Jataí', 10)
(29, 'Várzea Grande', 4)
(30, 'Maceió', 17)
(31, 'Salvador', 20)
(32, 'Macaé', 21)
(33, 'Cuiabá', 4)
(34, 'Santa Luzia do Itanhy', 18)
(35, 'João Pessoa', 22)
(36, 'Brusque', 23)
(37, 'Rio Grande', 11)
(38, 'Juazeiro do Norte', 24)
(39, 'Natal', 25)
(40, 'Londrina', 16)


In [27]:
# Transformando o caracter ',' em '.' na variável altura:
for i in range(0, len(content_df)):
    content_df.altura.iloc[i] = content_df.altura.iloc[i].replace(',', '.')

In [28]:
# Transformando a variável sexo para "M" e "F":
for i in range(0, len(content_df)):
    if content_df.sexo.iloc[i] == 'Masculino':
        content_df.sexo.iloc[i] = 'M'
    else:
        content_df.sexo.iloc[i] = 'F'

In [29]:
# Inserindo os dados na tabela tb_pessoa:
query_pessoa = "INSERT INTO tb_pessoa (nome, idade, data_nasc, sexo, signo, altura, peso, id_cidade, id_cor, id_tiposanguineo)\
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

for i in range(0, len(content_df)):
    data_pessoa = (content_df.nome.iloc[i], int(content_df.idade.iloc[i]), content_df.data_nasc.iloc[i], 
                   content_df.sexo.iloc[i], content_df.signo.iloc[i], float(content_df.altura.iloc[i]), 
                   float(content_df.peso.iloc[i]), int(content_df.cidade_id.iloc[i]), int(content_df.cor_id.iloc[i]), 
                   int(content_df.tipo_sanguineo_id.iloc[i]))
    print(data_pessoa)
    mycursor.execute(query_pessoa, data_pessoa)
    
mydb.commit()

('Silvana Hadassa', 56, '1964-12-06', 'F', 'Gêmeos', 1.67, 50.0, 0, 0, 0)
('Thales Kaique', 27, '1993-10-12', 'M', 'Sagitário', 1.63, 110.0, 1, 1, 1)
('Henry Nathan', 29, '1991-09-21', 'M', 'Virgem', 1.86, 106.0, 2, 2, 2)
('Lorenzo Vinicius', 64, '1956-10-22', 'M', 'Libra', 1.87, 100.0, 3, 0, 3)
('Cauã Henrique', 50, '1970-09-03', 'M', 'Peixes', 1.74, 83.0, 4, 0, 4)
('Francisco Thomas', 19, '2001-12-04', 'M', 'Áries', 1.91, 64.0, 5, 3, 2)
('Bianca Flávia', 71, '1949-01-23', 'F', 'Aquário', 1.54, 76.0, 1, 0, 5)
('Arthur Filipe', 33, '1987-12-23', 'M', 'Capricórnio', 1.91, 54.0, 6, 4, 2)
('Teresinha Jaqueline', 52, '1968-10-20', 'F', 'Libra', 1.84, 59.0, 7, 0, 2)
('Manuel Luís', 50, '1970-12-05', 'M', 'Touro', 1.81, 85.0, 8, 4, 6)
('Jorge Nicolas', 52, '1968-11-07', 'M', 'Câncer', 1.87, 54.0, 9, 4, 2)
('Ricardo Paulo', 37, '1983-12-16', 'M', 'Sagitário', 1.86, 52.0, 10, 0, 1)
('Gustavo Joaquim', 61, '1959-07-01', 'M', 'Capricórnio', 1.62, 106.0, 2, 2, 0)
('Isabel Yasmin', 50, '1970-06-22

<h1>Consultando o Banco de Dados</h1>

In [30]:
# Quantas pessoas possuem 50 ou mais anos de idade?
q1 = ("SELECT count(*) FROM tb_pessoa WHERE idade >= 50")
mycursor.execute(q1)

answ_1 = mycursor.fetchall()
for i in answ_1:
    print(i)

(30,)


In [31]:
# Qual é a média de idade das pessoas que moram em Mato Grosso?
q2 = ("SELECT sigla, avg(idade) FROM tb_pessoa \
        INNER JOIN tb_cidade ON tb_pessoa.id_cidade = tb_cidade.id \
        INNER JOIN tb_estado ON tb_cidade.id_estado = tb_estado.id \
        WHERE sigla = 'MT'")
mycursor.execute(q2)

answ_2 = mycursor.fetchall()
for i in answ_2:
    print(i)

('MT', Decimal('44.6000'))


In [32]:
# Quantas pessoas nasceram nos anos de 1968 a 1978?
q3 = ("SELECT count(*) FROM tb_pessoa \
        WHERE YEAR(data_nasc) BETWEEN 1968 AND 1978")
mycursor.execute(q3)

answ_3 = mycursor.fetchall()
for i in answ_3:
    print(i)

(14,)


In [33]:
# Qual é o nome da pessoa mais velha na base de dados?
q4 = ("SELECT nome, idade FROM tb_pessoa \
        WHERE data_nasc = (SELECT MIN(data_nasc) FROM tb_pessoa)")
mycursor.execute(q4)

answ_4 = mycursor.fetchall()
for i in answ_4:
    print(i)

('Thales Bento', 78)


In [34]:
# Qual é o nome da pessoa mais nova na base de dados?
q5 = ("SELECT nome, idade FROM tb_pessoa \
        WHERE data_nasc = (SELECT MAX(data_nasc) FROM tb_pessoa)")
mycursor.execute(q5)

answ_5 = mycursor.fetchall()
for i in answ_5:
    print(i)

('João Pedro', 18)


In [35]:
mydb.close()